In [68]:
import csv
import requests
import json
from geoconfig import google_api_key
import pandas as pd
import math as m
from math import sin, cos, atan2, sqrt, radians

In [69]:
# Read cities

cities = []
with open('installation_cities_cleaned.csv', 'rt',encoding="UTF8") as f:
    for row in csv.reader(f):
        city = row[0]
        if city != "NULL" and city != "city":
            cities.append(city)
print(cities)

['Aguila', 'Ajo', 'Anthem', 'Apache Junction', 'Arizona City', 'Arlington', 'Ash Fork', 'Avondale', 'Bagdad', 'Bellemont', 'Benson', 'Bisbee', 'Black Canyon', 'Bouse', 'Buckeye', 'Camp Verde', 'Carefree', 'Casa Grande', 'Cave Creek', 'Chandler', 'Chino Valley', 'Cibola', 'Clarkdale', 'Claypool', 'Congress', 'Coolidge', 'Cordes Lakes', 'Cornville', 'Cottonwood', 'Dateland', 'Desert Hills', 'Dewey-Humboldt', 'Douglas', 'Ehrenberg', 'El Mirage', 'Eloy', 'Flagstaff', 'Florence', 'Forest Lakes', 'Fort Mcdowell', 'Fountain Hills', 'Gadsden', 'Gila Bend', 'Gilbert', 'Glendale', 'Globe', 'Gold Canyon', 'Goodyear', 'Grand Canyon', 'Guadalupe', 'Happy Jack', 'Hayden', 'Hereford', 'Holbrook', 'Hotevilla', 'Humboldt', 'Jerome', 'Joseph City', 'Keams Canyon', 'Kearney', 'Kirkland', 'Lake Havasu City', 'Lake Montezuma', 'Las Brisas', 'Laveen', 'Litchfield Park', 'Littlefield', 'Littlefield Park', 'Maricopa', 'Mayer', 'Mesa', 'Miami', 'Morristown', 'Munds Park', 'Naco', 'New River', 'Northern Arizona

In [70]:
# Get weather stations

df_stations = pd.read_csv(r'../Weather/Weather.csv')
df_stations.rename(columns={'Latitude':'lng','Longitude':'lat'},inplace=True)
df_stations["Temp(F)"].astype("float")
stations_lng = df_stations["lat"]
num_stations = len(stations_lng)
stations_lat = df_stations["lng"]
stations_cities = df_stations["City"]
stations_temps = df_stations["Temp(F)"]
stations = []
for i in range(0, num_stations):
    station_location = {}
    station_location["station"] = stations_cities[i]
    station_location["lat"] = stations_lat[i]
    station_location["lng"] = stations_lng[i]
    station_location["temp"] = stations_temps[i]
    stations.append(station_location)
stations

[{'station': 'AJO', 'lat': 32.3698, 'lng': -112.8599, 'temp': 95.0},
 {'station': 'ALAMO DAM', 'lat': 34.228, 'lng': -113.5777, 'temp': 94.5},
 {'station': 'ALPINE', 'lat': 33.8492, 'lng': -109.1469, 'temp': 64.0},
 {'station': 'ANVIL RANCH', 'lat': 31.9793, 'lng': -111.3837, 'temp': 88.5},
 {'station': 'APACHE JUNCTION',
  'lat': 33.4625,
  'lng': -111.4813,
  'temp': 93.0},
 {'station': 'ARIZONA CITY', 'lat': 32.7306, 'lng': -111.6917, 'temp': 92.5},
 {'station': 'BAGDAD', 'lat': 34.5975, 'lng': -113.1745, 'temp': 83.0},
 {'station': 'BAKER BUTTE', 'lat': 34.46, 'lng': -111.41, 'temp': 68.5},
 {'station': 'BALDY', 'lat': 33.98, 'lng': -109.5, 'temp': 63.0},
 {'station': 'BAR M', 'lat': 34.86, 'lng': -111.61, 'temp': 71.5},
 {'station': 'BARTLETT DAM', 'lat': 33.8097, 'lng': -111.6497, 'temp': 90.0},
 {'station': 'BEAVER DAM', 'lat': 36.9139, 'lng': -113.9423, 'temp': 96.5},
 {'station': 'BEAVER HEAD', 'lat': 33.69, 'lng': -109.22, 'temp': 64.5},
 {'station': 'BEAVER SPRING', 'lat': 3

In [72]:
# Get temperature

def get_temp(stations,city_lat,city_lng):
    earth_radius_km = 6373
    earth_radius_mi = 3961
    c_lat = radians(city_lat)
    c_lng = radians(city_lng)
    smallest_distance = 4000
    city_temp = -9999
    nearest_station = {}
    #print(f"Looking for station closest to {c_lat},{c_lng}")
    for station in stations:
        s_lat = radians(station["lat"])
        s_lng = radians(station["lng"])
        s_name = station["station"]
        s_temp = station["temp"]
        d_lat = s_lat-c_lat
        d_lng = s_lng-c_lng
        a = (sin(d_lat/2))**2 + cos(c_lat) * cos(s_lat) * (sin(d_lng/2))**2
        c = 2 * atan2( sqrt(a), sqrt(1-a) )
        distance = earth_radius_mi*c
        #print(f"Checking {s_name} {distance} miles away")
        if distance < smallest_distance:
            smallest_distance = distance
            city_temp = s_temp
            nearest_station = station
    print(nearest_station)
    return city_temp

In [73]:
# Get latitude and longitude

locations = []
city_names = []
latitudes = []
longitudes = []
temperatures = []
base_url = "https://maps.googleapis.com/maps/api/geocode/json"
for city in cities:
    target_city = f"{city}, AZ"
    city_names.append(city)
    params = {"address": target_city, "key": google_api_key}
    response = requests.get(base_url, params=params)
    geo = response.json()
    lat = geo["results"][0]["geometry"]["location"]["lat"]
    latitudes.append(lat)
    lng = geo["results"][0]["geometry"]["location"]["lng"]
    longitudes.append(lng)
    temp = get_temp(stations,lat,lng)
    location = {"city":city,"lat":lat,"lng":lng,"temp":temp}
    locations.append(location)
print(locations)

{'station': 'SMITH PEAK ', 'lat': 34.1158, 'lng': -113.3472, 'temp': 95.0}
{'station': 'AJO', 'lat': 32.3698, 'lng': -112.8599, 'temp': 95.0}
{'station': 'DEER VALLEY', 'lat': 33.68833, 'lng': -112.08166999999999, 'temp': 95.5}
{'station': 'APACHE JUNCTION', 'lat': 33.4625, 'lng': -111.4813, 'temp': 93.0}
{'station': 'ARIZONA CITY', 'lat': 32.7306, 'lng': -111.6917, 'temp': 92.5}
{'station': 'PAINTED ROCK DAM', 'lat': 33.0772, 'lng': -113.0316, 'temp': 99.0}
{'station': 'WILLIAMS', 'lat': 35.2413, 'lng': -112.1929, 'temp': 71.5}
{'station': 'LITCHFIELD PARK', 'lat': 33.4991, 'lng': -112.363, 'temp': 98.5}
{'station': 'BAGDAD', 'lat': 34.5975, 'lng': -113.1745, 'temp': 83.0}
{'station': 'BELLEMONT WFO', 'lat': 35.2302, 'lng': -111.8221, 'temp': 65.0}
{'station': 'BENSON', 'lat': 31.8805, 'lng': -110.2403, 'temp': 83.0}
{'station': 'BISBEE', 'lat': 31.4475, 'lng': -109.9288, 'temp': 75.0}
{'station': 'SUNSET POINT ', 'lat': 34.1953, 'lng': -112.1417, 'temp': 87.5}
{'station': 'PARKER', '

{'station': 'SIERRA VISTA', 'lat': 31.536990000000003, 'lng': -110.28073, 'temp': 76.5}
{'station': 'IRON SPRINGS ', 'lat': 34.5853, 'lng': -112.5019, 'temp': 77.0}
{'station': 'SHOW LOW AIRPORT', 'lat': 34.2639, 'lng': -110.0075, 'temp': 70.0}
{'station': 'YUMA MCAS - COOP', 'lat': 32.65, 'lng': -114.61667, 'temp': 98.5}
{'station': 'CORDES', 'lat': 34.305, 'lng': -112.165, 'temp': 83.0}
{'station': 'PAYSON ', 'lat': 34.2431, 'lng': -111.3028, 'temp': 82.0}
{'station': 'ST JOHNS INDUSTRIAL AIR PARK', 'lat': 34.51833, 'lng': -109.37917, 'temp': 81.0}
{'station': 'BAKER BUTTE', 'lat': 34.46, 'lng': -111.41, 'temp': 68.5}
{'station': 'YOUNGTOWN', 'lat': 33.595, 'lng': -112.3014, 'temp': 97.0}
{'station': 'YOUNGTOWN', 'lat': 33.595, 'lng': -112.3014, 'temp': 97.0}
{'station': 'TEMPE', 'lat': 33.4258, 'lng': -111.9216, 'temp': 96.0}
{'station': 'APACHE JUNCTION', 'lat': 33.4625, 'lng': -111.4813, 'temp': 93.0}
{'station': 'YOUNGTOWN', 'lat': 33.595, 'lng': -112.3014, 'temp': 97.0}
{'statio

In [74]:
locations_df = pd.DataFrame(locations)
locations_df.head()

,city,lat,lng,temp
0,Aguila,33.942807,-113.174081,95.0
1,Ajo,32.371725,-112.860710,95.0
2,Anthem,33.865232,-112.135734,95.5
3,Apache Junction,33.415048,-111.549578,93.0
4,Arizona City,32.755893,-111.670958,92.5


In [75]:
locations_df.to_csv(r'cities_lat_lng_tmp.csv')